<a href="https://colab.research.google.com/github/PearlSikka/language-ninja/blob/master/Training_Language_models_on_TPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import os
import pathlib

# Upload the API token.
def get_kaggle():
  try:
    import kaggle
    return kaggle
  except OSError:
    pass

  token_file = pathlib.Path("~/.kaggle/kaggle.json").expanduser()
  token_file.parent.mkdir(exist_ok=True, parents=True)

  try:
    from google.colab import files
  except ImportError:
    raise ValueError("Could not find kaggle token.")

  uploaded = files.upload()
  token_content = uploaded.get('kaggle.json', None)
  if token_content:
    token_file.write_bytes(token_content)
    token_file.chmod(0o600)
  else:
    raise ValueError('Need a file named "kaggle.json"')
  
  import kaggle
  return kaggle


kaggle = get_kaggle()

In [46]:
!kaggle datasets download -d abhishek/bertsrc

bertsrc.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
!unzip /content/bertsrc.zip

Archive:  /content/bertsrc.zip
replace CONTRIBUTING.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace create_pretraining_data.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: create_pretraining_data.py  
replace extract_features.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: extract_features.py     
replace modeling.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: modeling.py             
replace modeling_test.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: modeling_test.py        
replace multilingual.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: multilingual.md         
replace optimization.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: optimization.py         
replace optimization_test.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: optimizati

In [48]:
!pip install -U tokenizers

y
Requirement already up-to-date: tokenizers in /usr/local/lib/python3.7/dist-packages (0.10.1)


In [49]:
!pip install tensorflow==1.15

In [50]:
import tokenizers

In [51]:
#create vocabulary from corpus

#bert word piece tokenizer

bwpt= tokenizers.BertWordPieceTokenizer(vocab=None, clean_text=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##')

In [52]:
!kaggle datasets download -d abhishek/hindi-oscar-corpus

hindi-oscar-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)


In [53]:
!unzip /content/hindi-oscar-corpus.zip

Archive:  /content/hindi-oscar-corpus.zip
replace hi_dedup.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hi_dedup.txt            
replace hi_dedup_1000.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hi_dedup_1000.txt       


In [54]:
!head /content/hi_dedup_1000.txt

शारदा पारा के मिलन चैiक से आज महापौर देवेन्द्र यादव, पार्षद छोटे लाल चैधरी के साथ वार्ड का भ्रमण करते हुए वार्ड की पेयजल, स्वच्छता सफाई, विकास कार्यों का निरीक्षण किया।
सूचना का अधिकार - विभाग द्वारा तैयार 17 column सम्बंधित पंजी ,कार्यालय नगर पालिक निगम भिलाई जोन-06 रिसाली
आज सम्पत्तिकर अधिकारी एच.के. चन्द्राकर ने महाप्रबंधक भारत संचार निगम लिमिटेड दुर्ग को सम्पत्तिकर की राशि कुर्कीवारण्ट जारी कर दिया, कुर्कीवारण्ट के साथ अधिभार सहित राशि वसुल किये जाने की नोटिस तामिल की गई है।
राष्ट्रपति, राज्यपाल, मुख्यमंत्री, जैसे संवैधानिक पदों पर आज महिलाएं देश के विकास के लिए निर्णय ले रहीं है।
परसा कोल ब्लॉक जनसुनवाई में अडानी द्वारा धनबल का दुरुपयोग : कम्पनी की रिपोर्ट फ़र्ज़ी भृमात्मक और झूटी – छत्तीसगढ़ बॉस्केट
परसा कोल ब्लॉक जनसुनवाई में अडानी द्वारा धनबल का दुरुपयोग : कम्पनी की रिपोर्ट फ़र्ज़ी भृमात्मक और झूटी
पर्यावरण एक ऐसा विषय हैं जिसका प्रभाव आने वाली पीढ़ियों पर भी पड़ता हैं परन्तु खनन और उधोगिकरण कि अंधी दौड़ में नियमो और कानूनों का पालन करवाने वाला प्रशासनिक तंत्र स्वयं विधि विरुद्ध तरीके 

In [55]:
bwpt.train(files=["/content/hi_dedup_1000.txt"], vocab_size=30000,
           min_frequency=3,
           limit_alphabet=1000, 
           special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]'] )

In [56]:
mkdir /content/output

mkdir: cannot create directory ‘/content/output’: File exists


In [57]:
bwpt.save('/content/output/hindi.txt')

In [58]:
!head /content/output/hindi.txt

{"version":"1.0","truncation":null,"padding":null,"added_tokens":[{"id":0,"special":true,"content":"[PAD]","single_word":false,"lstrip":false,"rstrip":false,"normalized":false},{"id":1,"special":true,"content":"[UNK]","single_word":false,"lstrip":false,"rstrip":false,"normalized":false},{"id":2,"special":true,"content":"[CLS]","single_word":false,"lstrip":false,"rstrip":false,"normalized":false},{"id":3,"special":true,"content":"[MASK]","single_word":false,"lstrip":false,"rstrip":false,"normalized":false},{"id":4,"special":true,"content":"[SEP]","single_word":false,"lstrip":false,"rstrip":false,"normalized":false}],"normalizer":{"type":"BertNormalizer","clean_text":true,"handle_chinese_chars":true,"strip_accents":true,"lowercase":true},"pre_tokenizer":{"type":"BertPreTokenizer"},"post_processor":null,"decoder":{"type":"WordPiece","prefix":"##","cleanup":true},"model":{"type":"WordPiece","unk_token":"[UNK]","continuing_subword_prefix":"##","max_input_chars_per_word":100,"vocab":{"[PAD]"

In [59]:
!kaggle datasets download -d abhishek/bert-base-uncased 

bert-base-uncased.zip: Skipping, found more recently modified local copy (use --force to force download)


In [60]:
!unzip /content/bert-base-uncased.zip -d bertuncased

Archive:  /content/bert-base-uncased.zip
replace bertuncased/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: bertuncased/config.json  
replace bertuncased/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: bertuncased/pytorch_model.bin  
replace bertuncased/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: bertuncased/vocab.txt   


In [61]:
!python create_pretraining_data.py \
    --input_file=/content/hi_dedup_1000.txt \
    --output_file=/content/output/tf_examples.tfrecord \
    --vocab_file=/content/output/hindi.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5






W0307 14:35:24.362984 140517552355200 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0307 14:35:24.363240 140517552355200 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0307 14:35:24.363511 140517552355200 module_wrapper.py:139] From /content/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0307 14:35:24.365427 140517552355200 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0307 14:35:24.366968 140517552355200 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***
I0307 14:35:24.367153 14

In [62]:
!head /content/output/tf_examples.tfrecord